# 1.- Read IC 348 & NGC 1333 catalogues from Luhman, Esplin & Loutrel 2016
### This catalogue includes most recent candidate members and supersedes the Luhman+2003 & Lada+2006 catalogues
### https://ui.adsabs.harvard.edu/abs/2016ApJ...827...52L/abstract

In [1]:
import warnings, os
from astropy                  import units as u
from astropy.table            import Table, hstack, join
from astropy.utils.exceptions import AstropyWarning
from astropy.coordinates      import SkyCoord
import library_initial        as lib_ini

In [2]:
# Dedicated functions ==================================
def read_tb(inp_tb, cols, label = ''):
    """
    Read input catalogue tables from Luhman 2016. Adds coordinates column.
    """
    tab = Table.read(inp_tb, format = 'ascii')
    tab = tab[cols]
    
    tab['coords'] = [str(tab['RAh'][i]) + ' ' + str(tab['RAm'][i]) + ' ' + str(tab['RAs'][i]) + ' ' + str(tab['DE-'][i]) +
                     str(tab['DEd'][i]) + ' ' + str(tab['DEm'][i]) + ' ' + str(tab['DEs'][i]) for i in range(len(tab))]
    
    inp_coords   = SkyCoord(tab['coords'], unit = (u.hourangle, u.deg))
    tab.remove_columns(['RAh', 'RAm', 'RAs', 'DE-', 'DEd', 'DEm', 'DEs'])
    tab['RAdeg'] = inp_coords.ra.deg
    tab['DEdeg'] = inp_coords.dec.deg
    tab.label = label
    for col in ['RAdeg', 'DEdeg']:
        tab[col].format = '8.5f'
        tab[col].unit   = u.degree        
    
    return tab

In [3]:
# Read & prepare tables ================================
warnings.filterwarnings('ignore', category=AstropyWarning, append=True)
cols        = ['Name', 'RAh', 'RAm', 'RAs', 'DE-', 'DEd', 'DEm', 'DEs', 'Adopt', 'pmRA', 'e_pmRA', 'pmDE', 'e_pmDE']
cat_ic348   = read_tb('table_1.dat', cols = cols + ['Luhman'], label = 'IC_348')
cat_ngc1333 = read_tb('table_2.dat', cols = cols,              label = 'NGC_1333')

In [4]:
# Cross-match catalogues against 2MASS =================
inp_cats = [cat_ngc1333, cat_ic348]
out_cats = []

for inp_cat in inp_cats:
    inp_coords     = SkyCoord(inp_cat['RAdeg'],inp_cat['DEdeg'])
    inp_cat_tmasss = lib_ini.query_coords(inp_coords, catalog = "II/246", radius=1.0 * u.arcsecond, verbose = False)

    # Combine input & 2mass catalogues =================
    merged       = hstack([inp_cat, inp_cat_tmasss])
    merged       = merged[merged['_2MASS'] != '0']
    merged.remove_columns(['coords', 'RAdeg', 'DEdeg']) # Remove redundant Columns
    merged.label = inp_cat.label
    
    out_cats.append(merged)

In [5]:
# Save Catalogues ======================================
for i in range(2):

    name_out_0 = 'sample_ini_' + inp_cats[i].label + '.vot'   
    name_out_1 = 'sample_ini_' + out_cats[i].label + '_2MASS.vot'

    inp_cats[i].write(name_out_0, format = 'votable', overwrite = True)
    out_cats[i].write(name_out_1, format = 'votable', overwrite = True)

    print(f'Saving Input   sample as: {name_out_0:>30s}. N_elements ={len(inp_cats[i]):>4.0f}')    
    print(f'Saving Initial sample as: {name_out_1:>30s}. N_elements ={len(out_cats[i]):>4.0f}')
    os.system('cp ' + name_out_1 + ' ../sample_analysis/')
    print()

print()
print('INITIAL SAMPLES SAVED IN ../sample_analysis/ DIRECTORY')

Saving Input   sample as:        sample_ini_NGC_1333.vot. N_elements = 203
Saving Initial sample as:  sample_ini_NGC_1333_2MASS.vot. N_elements = 160

Saving Input   sample as:          sample_ini_IC_348.vot. N_elements = 478
Saving Initial sample as:    sample_ini_IC_348_2MASS.vot. N_elements = 416


INITIAL SAMPLES SAVED IN ../sample_analysis/ DIRECTORY


In [6]:
ls -1

01_make_sample_initial.ipynb
README.md
__pycache__/
library_initial.py
sample_ini_IC_348.vot
sample_ini_IC_348_2MASS.vot
sample_ini_NGC_1333.vot
sample_ini_NGC_1333_2MASS.vot
table_1.dat
table_2.dat
